## Fetch Rewards: Solutions Engineer - Take Home Challenge

**Work done in this file:**

1. Review unstructured JSON data and diagram a new structured relational data model
    1. Imported JSON files one by one and used custom code to read those and convert to dictionary/csv.
    2. Checked for duplicates & Null Values
    3. Mentioned the **(.drawio)** file to refer for relational data model
             
    
2. Generate queries that answers a predetermined business question    
    
       Mentioned the .sql file to refer for queries answering below business questions  
    
    1. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
    2. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
    3. Which brand has the most spend among users who were created within the past 6 months?
    4. Which brand has the most transactions among users who were created within the past 6 months?
    
3. Evaluate Data Quality Issues in the Data Provided

       Mentioned .sql file identifying some of the data quality issues
       
4. Communicate with Stakeholders
        
        Mentioned the .txt file containing email communication to be sent to stakeholder
        
> SQL Dialect used: MS SQL Server (SQL Server Management Studio 16.0.19061.0


### 1.  Review unstructured JSON data and diagram a new structured relational data model

**Note:** 
Provided JSON files contain nested dictionaries. Instead of using read_json function (pandas) directly, I developed custom code to read dictionaries from the json files which allowed more control over extracting and modifying the nested data.

**Understanding:**

There were total 3 JSON files (receipts, brands, users) provided. 

Receipts: 
> On analyzing the receipts.json file, I noticed that some of the keys 'id', date fields, 'rewardReceiptItemList' have dictionary as its values. 

> Extracting id & date fields was straight-forward as compared to 'rewardReceiptItemList'. Created a different dataframe/table for rewardReceiptItemList. 

> Formatted the date fields from epoch time format(Unix Time stamp) to readable format.

> There exists one to many relationship between Receipts and rewardReceiptItemList with primary and foreign key as 'id' in both tables.


Brands: 
> Value for 'cpg' key was a list with oid & ref, hence created two new columns in brands as cpg_id, cpg_ref. 

> There exists many to many relationship between 'brands' and 'rewardReceiptItemList'. 


Users: 
> Formatted the date fields from epoch time format(Unix Time stamp) to readable format.

> There exists one to many relationship between 'users' and 'receipts'


In [1]:
import pandas as pd
from datetime import datetime
import json

In [2]:
# Function for loading json file

def load_data(file_name):
    data = []
    with open(file_name, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [3]:
# Function for modifying date format

def modify_date(dt_value):
        dt=dt_value/ 1000  # Convert milliseconds to seconds
        date_val = datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
        return date_val

##### Receipts:

In [4]:
#Importing receipts.json file

receipt_data = load_data('receipts.json')

In [5]:
# Formatting _id & date keys 

receipt_data_new=[]

for i in range(len(receipt_data)):
    receipt1={}
    for key,value in receipt_data[i].items():
        
        if ('Date' in key) or ('date'in key):
            #dt = value['$date']/1000  # Convert milliseconds to seconds
            #date_val = datetime.utcfromtimestamp(dt).strftime('%Y-%m-%d %H:%M:%S')
            receipt1[key] = modify_date(value['$date'])          
        elif (key == '_id'):
            receipt1[f'{key}']=value['$oid']
            
        else:
            receipt1[f'{key}'] = value
    receipt_data_new.append(receipt1)

In [6]:
# Receipts dataframe

_schema=['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'pointsEarned', 'purchaseDate', 'purchasedItemCount',  'rewardsReceiptStatus', 'totalSpent', 'userId']

receipts_df = pd.DataFrame(receipt_data_new,columns=_schema)

In [7]:
receipts_df.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaN,2021-01-03 15:25:42,NaN,5,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [8]:
receipts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    object 
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptStatus     1119 non-null   object 
 12  totalSpent               684 non-null    object 
 13  userId                   1119 non-null   object 
dtypes: float64(2), object(12

In [9]:
receipts_df.duplicated().sum()

0

In [10]:
# % Null values in receipts
round(receipts_df.isnull().mean()*100,2).sort_values()

_id                         0.00
createDate                  0.00
dateScanned                 0.00
modifyDate                  0.00
rewardsReceiptStatus        0.00
userId                      0.00
totalSpent                 38.87
purchaseDate               40.04
purchasedItemCount         43.25
pointsEarned               45.58
finishedDate               49.24
bonusPointsEarned          51.39
bonusPointsEarnedReason    51.39
pointsAwardedDate          52.01
dtype: float64

In [11]:
receipts_df.to_csv(r'Receipts.csv', index=False)

##### Reward Receipt Item List:

In [12]:
# Reward Receipt items List 

reward_receipt_new=[]

for i in range(len(receipt_data)):   
    if 'rewardsReceiptItemList' in receipt_data[i].keys():        
        for k in receipt_data[i]['rewardsReceiptItemList']:
            reward_receipt1={}
            for key,value in k.items():
                if ('Date' in key) or ('date'in key):
                    reward_receipt1[f'{key}'] = modify_date(value) # date formatting
                else:
                    reward_receipt1['_id'] = receipt_data[i]['_id']['$oid'] #extracting _id
                    reward_receipt1[f'{key}'] = value
            reward_receipt_new.append(reward_receipt1)

In [13]:
# Reward Receipt Item list dataframe

reward_receipts_df =pd.DataFrame(reward_receipt_new)


In [14]:
reward_receipts_df.head()

,_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
reward_receipts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   _id                                 6941 non-null   object 
 1   barcode                             3090 non-null   object 
 2   description                         6560 non-null   object 
 3   finalPrice                          6767 non-null   object 
 4   itemPrice                           6767 non-null   object 
 5   needsFetchReview                    813 non-null    object 
 6   partnerItemId                       6941 non-null   object 
 7   preventTargetGapPoints              358 non-null    object 
 8   quantityPurchased                   6767 non-null   float64
 9   userFlaggedBarcode                  337 non-null    object 
 10  userFlaggedNewItem                  323 non-null    object 
 11  userFlaggedPrice                    299 non

In [16]:
reward_receipts_df.columns

Index(['_id', 'barcode', 'description', 'finalPrice', 'itemPrice',
       'needsFetchReview', 'partnerItemId', 'preventTargetGapPoints',
       'quantityPurchased', 'userFlaggedBarcode', 'userFlaggedNewItem',
       'userFlaggedPrice', 'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

In [17]:
reward_receipts_df.duplicated().sum()

0

In [18]:
# % Null values in reward_receipts
round(reward_receipts_df.isnull().mean()*100,2).sort_values()

_id                                    0.00
partnerItemId                          0.00
finalPrice                             2.51
itemPrice                              2.51
quantityPurchased                      2.51
description                            5.49
discountedItemPrice                   16.89
originalReceiptItemText               17.01
barcode                               55.48
brandCode                             62.54
rewardsProductPartnerId               67.31
rewardsGroup                          75.06
pointsPayerId                         81.75
priceAfterCoupon                      86.23
pointsEarned                          86.64
metabriteCampaignId                   87.57
needsFetchReview                      88.29
competitiveProduct                    90.71
targetPrice                           94.55
preventTargetGapPoints                94.84
pointsNotAwardedReason                95.10
userFlaggedBarcode                    95.14
userFlaggedNewItem              

In [19]:
reward_receipts_df.to_csv(r'RewardReceiptItems.csv', index=False)

##### Brands:

In [20]:
# Importing brands.json

brands_data = load_data('brands.json')

In [21]:
#Brands df

brands_new=[]

for i in range(len(brands_data)):
    brands_1={}
    for key,value in brands_data[i].items():
        if (key=='cpg'):
            brands_1[f'{key}_id']=value['$id']['$oid']
            brands_1[f'{key}_ref']=value['$ref']            
        elif (key=='_id'):
            brands_1[f'{key}']=value['$oid']            
        else:
            brands_1[f'{key}'] = value
    brands_new.append(brands_1)


In [22]:
# Brands dataframe

brands_df =pd.DataFrame(brands_new)

In [23]:
brands_df.head()

,_id,barcode,category,categoryCode,cpg_id,cpg_ref,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,Cogs,test brand @1612366101024,False,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Cogs,Starbucks,False,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,Cogs,test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,Cogs,test brand @1612366146827,False,TEST BRANDCODE @1612366146827


In [24]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           1167 non-null   object
 1   barcode       1167 non-null   object
 2   category      1012 non-null   object
 3   categoryCode  517 non-null    object
 4   cpg_id        1167 non-null   object
 5   cpg_ref       1167 non-null   object
 6   name          1167 non-null   object
 7   topBrand      555 non-null    object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


In [25]:
brands_df.duplicated().sum()

0

In [26]:
brands_df.columns

Index(['_id', 'barcode', 'category', 'categoryCode', 'cpg_id', 'cpg_ref',
       'name', 'topBrand', 'brandCode'],
      dtype='object')

In [27]:
# % Null values in brands 
round(brands_df.isnull().mean()*100,2).sort_values()

_id              0.00
barcode          0.00
cpg_id           0.00
cpg_ref          0.00
name             0.00
category        13.28
brandCode       20.05
topBrand        52.44
categoryCode    55.70
dtype: float64

In [28]:
brands_df.to_csv(r'Brands.csv', index=False)

##### Users:

In [29]:
# Importing users.json 

users_data = load_data('users.json')

In [30]:
users_data_new=[]

for i in range(len(users_data)):
    users1={}
    for key,value in users_data[i].items():    
        if (key == 'lastLogin') or (key == 'createdDate'):
            users1[key] = modify_date(value['$date'])          
        elif (key == '_id'):
            users1[f'{key}']=value['$oid']        
        else:
            users1[f'{key}'] = value
    users_data_new.append(users1)

In [31]:
# Users Dataframe

users_df =pd.DataFrame(users_data_new)

In [32]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


In [33]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


In [34]:
#Checking for Duplicates

users_df.duplicated().sum()

283

In [35]:
len(users_df)

495

In [36]:
users_df = users_df.drop_duplicates()

In [37]:
len(users_df)

212

In [38]:
# % Null Values in Users
round(users_df.isnull().mean()*100,2).sort_values()

_id              0.00
active           0.00
createdDate      0.00
role             0.00
signUpSource     2.36
state            2.83
lastLogin       18.87
dtype: float64

In [39]:
users_df.to_csv(r'Users.csv', index=False)

**Data Model for the tables:**

> Attached in FetchRewardsAssessment.drawio file attached in the repository.

### 2.  Generate a query that answers a predetermined business question

> Attached FetchRewardsAssessment.sql file for this section

### 3. Evaluate Data Quality Issues in the Data Provided

> Attached FetchRewardsAssessment_DQ.sql file for this section

### 4. Communicate with Stakeholders

> Attached Email_Communication.txt file for this section